In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [16]:
cap = cv2.VideoCapture("video.MP4")

while True:
    ret,frame = cap.read()
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    gray = gray[250:650,:]
    
    #print(gray.shape)
    
    #blur1 = cv2.blur(gray,(5,5)) 
    blur2 = cv2.medianBlur(gray,9)
    blur3 = cv2.GaussianBlur(blur2,(7,7),0)
    
    ret,th1 = cv2.threshold(blur3,190,255,cv2.THRESH_TOZERO_INV)
    ret,th1 = cv2.threshold(th1,110,255,cv2.THRESH_BINARY)
    #th2 = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,7,2)
    
    kernel = np.ones((5,5),np.uint8)
    #erosion = cv2.erode(th1,kernel,iterations=1)
    
    #closing = cv2.morphologyEx(dilation,cv2.MORPH_CLOSE,kernel)
    opening = cv2.morphologyEx(th1,cv2.MORPH_OPEN,kernel)
    dilation = cv2.dilate(opening,kernel,iterations=1)
    
    black = np.zeros((400,1280))
    edges = cv2.Canny(dilation,240,260)
    lines = cv2.HoughLinesP(dilation,1,np.pi/180,30,minLineLength=40,maxLineGap=80)
    
    #print(lines.shape)
    
    if lines is not None:     #Error - NoneType obj is not iterable
        for line in lines:
            x1,y1,x2,y2 = line[0]
            cv2.line(black,(x1,y1),(x2,y2),(255,255,255),2)
            
    dilate_2 = cv2.dilate(dilation,kernel,iterations=1)
    opening_2 = cv2.morphologyEx(dilate_2, cv2.MORPH_OPEN,kernel)
    
    #print(opening_2.shape)
    
    img = opening_2
    white_pts = []
    mid_pts = []
    final_mid = []
    
    for j in range(50):
        for i in range(1280):
            if img[300+j][i] == 255:
                white_pts.append(i)
            try:    
                x1 = min(white_pts)
                x2 = max(white_pts)
                mid_pts.append((x1+x2)/2)
            except:
                mid_pts.append(640)
                
    mid = int(sum(mid_pts)/len(mid_pts))
    final_mid.append(mid)
    cv2.circle(img,(mid,300),15,(255,255,255),1)
    
    print(len(final_mid))
    #cv2.imshow("Thresh",th1)
    #cv2.imshow("Dilation",dilate_2)
    #cv2.imshow("Closing",closing)
    #cv2.imshow("Opening",opening_2)
    #cv2.imshow("AdaptiveThresh",th2)
    #cv2.imshow("Original",frame)
    
    cv2.imshow("Final",img)
    
    key_pressed = cv2.waitKey(1) & 0xFF
    if(key_pressed == ord('q')):
        break
    
cap.release()
cv2.destroyAllWindows()

1
